In [1]:
import tensorflow as tf
print(tf.__version__)

2.3.0


In [2]:
import tensorflow_datasets as tfds
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete00XDKY/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete00XDKY/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete00XDKY/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [3]:
info

tfds.core.DatasetInfo(
    name='imdb_reviews',
    version=1.0.0,
    description='Large Movie Review Dataset.
This is a dataset for binary sentiment classification containing substantially more data than previous benchmark datasets. We provide a set of 25,000 highly polar movie reviews for training, and 25,000 for testing. There is additional unlabeled data for use as well.',
    homepage='http://ai.stanford.edu/~amaas/data/sentiment/',
    features=FeaturesDict({
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
        'text': Text(shape=(), dtype=tf.string),
    }),
    total_num_examples=100000,
    splits={
        'test': 25000,
        'train': 25000,
        'unsupervised': 50000,
    },
    supervised_keys=('text', 'label'),
    citation="""@InProceedings{maas-EtAl:2011:ACL-HLT2011,
      author    = {Maas, Andrew L.  and  Daly, Raymond E.  and  Pham, Peter T.  and  Huang, Dan  and  Ng, Andrew Y.  and  Potts, Christopher},
      title     = {Learning Word

In [10]:
import numpy as np

train_data, test_data = imdb['train'], imdb['test']

In [60]:
X_train = []
y_train = []

X_test = []
y_test = []

for x, y in train_data:
  X_train.append(x.numpy().decode("utf-8"))
  y_train.append(y.numpy())

for x, y in test_data:
  X_test.append(x.numpy().decode("utf-8"))
  y_test.append(y.numpy())

y_train = np.array(y_train)
y_test = np.array(y_test)

print('X_train : ', len(X_train))
print('y_train : ', len(y_train), '\n')
print('X_test  : ', len(X_test))
print('y_test  : ', len(y_test))

X_train :  25000
y_train :  25000 

X_test  :  25000
y_test  :  25000


In [65]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type='post'
oov_tok = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(X_train)
train_sequences = tokenizer.texts_to_sequences(X_train)
test_sequences = tokenizer.texts_to_sequences(X_test)
padded_train_sequences = pad_sequences(train_sequences, maxlen=max_length, truncating=trunc_type)
padded_test_sequences = pad_sequences(test_sequences, maxlen=max_length, truncating=trunc_type)

In [66]:
from tensorflow.keras.layers import Embedding, Flatten, Dense, Input

class MyModel(tf.keras.Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.embedding = Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length)
    self.flatten = Flatten()
    self.fc1 = Dense(128, activation='relu')
    self.fc2 = Dense(1, activation='sigmoid')

  def call(self, inputs):
    x = self.embedding(inputs)
    x = self.flatten(x)
    x = self.fc1(x)
    x = self.fc2(x)
    return x
  
  def model(self):
        x = Input([max_length])
        return tf.keras.Model(inputs=[x], outputs=self.call(x))

In [67]:
model = MyModel()
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.model().summary()

Model: "functional_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        [(None, 120)]             0         
_________________________________________________________________
embedding_12 (Embedding)     (None, 120, 16)           160000    
_________________________________________________________________
flatten_12 (Flatten)         (None, 1920)              0         
_________________________________________________________________
dense_24 (Dense)             (None, 128)               245888    
_________________________________________________________________
dense_25 (Dense)             (None, 1)                 129       
Total params: 406,017
Trainable params: 406,017
Non-trainable params: 0
_________________________________________________________________


In [68]:
num_epochs=10
model.fit(padded_train_sequences, y_train, epochs=num_epochs, validation_data=(padded_test_sequences, y_test))

Epoch 1/10
782/782 [==============================] - 6s 8ms/step - loss: 0.4584 - accuracy: 0.7673 - val_loss: 0.3869 - val_accuracy: 0.8223
Epoch 2/10
782/782 [==============================] - 6s 7ms/step - loss: 0.1914 - accuracy: 0.9281 - val_loss: 0.4931 - val_accuracy: 0.7998
Epoch 3/10
782/782 [==============================] - 6s 7ms/step - loss: 0.0360 - accuracy: 0.9895 - val_loss: 0.7651 - val_accuracy: 0.7960
Epoch 4/10
782/782 [==============================] - 6s 7ms/step - loss: 0.0055 - accuracy: 0.9990 - val_loss: 0.9442 - val_accuracy: 0.7996
Epoch 5/10
782/782 [==============================] - 6s 7ms/step - loss: 0.0013 - accuracy: 0.9998 - val_loss: 1.0215 - val_accuracy: 0.8006
Epoch 6/10
782/782 [==============================] - 6s 7ms/step - loss: 2.3384e-04 - accuracy: 1.0000 - val_loss: 1.0755 - val_accuracy: 0.8004
Epoch 7/10
782/782 [==============================] - 6s 7ms/step - loss: 1.1301e-04 - accuracy: 1.0000 - val_loss: 1.1177 - val_accuracy: 0.801

In [80]:
test_text = 'This movie is really great!.'
sequence = tokenizer.texts_to_sequences([test_text])
padded = pad_sequences(sequence, maxlen=max_length, truncating=trunc_type)
pred = model(padded).numpy()[0,0] 
if pred > 0.5:
  print('Review : ', test_text)
  print('Prediction is Good review at confident : ', round(pred, 4))
else:
  print('Review : ', test_text)
  print('Prediction is Bad review at confident : ', round(1-pred, 4))

print()

test_text = 'This movie is suck. I dont want to see this movie again.'
sequence = tokenizer.texts_to_sequences([test_text])
padded = pad_sequences(sequence, maxlen=max_length, truncating=trunc_type)
pred = model(padded).numpy()[0,0] 
if pred > 0.5:
  print('Review : ', test_text)
  print('Prediction is Good review at confident : ', round(pred, 4))
else:
  print('Review : ', test_text)
  print('Prediction is Bad review at confident  : ', round(1-pred, 4))

Review :  This movie is really great!.
Prediction is Good review at confident :  0.7595

Review :  This movie is suck. I dont want to see this movie again.
Prediction is Bad review at confident  :  0.9527
